In [9]:
# import the necessary packages
import librosa
import librosa.display
import pandas as pd
import numpy as np
#import glob
from glob import glob
import cv2
import os
import matplotlib.pyplot as pltimport 


%matplotlib inline
from timeit import default_timer as timer


# **Making CSV Files**

In [10]:
def write_to_csv(csvName, dataDir):
    with open(csvName,"w") as file:
        file.write("ID,Class\n")
        for fn in dataDir:
            fileID  = fn.split('/')[-1].split('.')[0]
            classID = fileID.split('-')[0]
            file.write(str(fileID)+','+classID)
            file.write('\n')

In [11]:
######### Making .csv files #############

#train_dir = np.array(glob("ALL_A_TRAIN/*"))####記得改
#write_to_csv("train.csv",train_dir)####記得改

In [12]:
######### Making .csv files #############

#test_dir = np.array(glob("ALL_A_TEST/*"))####記得改
#write_to_csv("test.csv",test_dir)####記得改

## Functions for Feature extraction & Model Creation

In [13]:
# Testing the extraction of MFCC & others using Librosa
filename = '1-3.wav'

import os
leo= os.getcwd()

print ('',leo)
X, sample_rate = librosa.load(filename) 

start = timer()#####TONY

#mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0) #40######
#mfcc_delta = librosa.feature.delta(mfccs) #TONY
#mfcc_delta2 = librosa.feature.delta(mfccs, order=2)#TONY

#chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
#contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)
#tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X), sr=sample_rate).T,axis=0) 

#ext_features = np.hstack([mfccs,mfcc_delta, mfcc_delta2])

end = timer()#######TONY
print("start: ", start)
print("end: ", end)
print("time taken: {0} minutes {1:.1f} seconds".format((end - start), (end - start)%60))#######TONY

#print(ext_features.shape)

 C:\Users\Leo\Desktop\交接\CNN
start:  337.6198497
end:  337.6262605
time taken: 0.006410800000026029 minutes 0.0 seconds


In [14]:
#ext_features

In [15]:
#npf = np.array(ext_features)

In [16]:
#npf.shape

**Feature extraction & Spec Images**

In [17]:
# Extracting MFCC
def extract_features(file_name):
#extract_features(file_name):
    try:
    
   
        X, sample_rate = librosa.load(file_name, res_type='kaiser_fast')
        stft = np.abs(librosa.stft(X))
        
        #mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        #mfccsscaled = np.mean(mfccs.T,axis=0)
        
        mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0) #40######
        mfcc_delta = librosa.feature.delta(mfccs) #TONY
        mfcc_delta2 = librosa.feature.delta(mfccs, order=2)#TONY
        
        chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
        mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
        contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)
        tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X), sr=sample_rate).T,axis=0)
    
        ###### ADD NEW FEATURES (SPECTRAL RELATED)##### 24-SEP
        cent = np.mean(librosa.feature.spectral_centroid(y=X, sr=sample_rate).T,axis=0)
        flatness = np.mean(librosa.feature.spectral_flatness(y=X).T,axis=0)
        #rolloff = np.mean(librosa.feature.spectral_rolloff(y=X, sr=sample_rate).T,axis=0)
        rolloff = np.mean(librosa.feature.spectral_rolloff(S=stft, sr=sample_rate).T,axis=0)
        rms = np.mean(librosa.feature.rms(S=stft).T,axis=0)
        
        #ext_features = np.hstack([mfccs,mfcc_delta, mfcc_delta2,mel,rolloff])
        ext_features = np.hstack([mfccs,mfcc_delta, mfcc_delta2, chroma, mel, contrast, tonnetz, cent,flatness, rolloff,rms])
        
        
    except Exception as e:
        print("Error encountered while parsing file: ", file_name)
        return None 
     
    return np.array(ext_features)

In [18]:
import wave
import cv2
#f = wave.open(r"C:\Users\Leo\Desktop\MSP-310-LAB\HO-To-Labmates\EXPERIMENT_NN\ALL_A_TRAIN\01-01.wav",'rb')
testFile ='1-3.wav'
#start = timer()#####TONY
#f = wave.open(file,'rb')
testF = extract_features(testFile)

end = timer()#######TONY
print("start: ", start)
print("end: ", end)
print("time taken: {0} minutes {1:.1f} seconds".format((end - start), (end - start)%60))#######TONY
testF.shape

start:  337.6198497
end:  340.1706341
time taken: 2.5507843999999977 minutes 2.6 seconds


(277,)

In [19]:
# Extracting Spectrogram Features
def extract_features_spec(file_name):
   
    try:
        X, sample_rate = librosa.load(file_name, res_type='kaiser_fast')
        stft = np.abs(librosa.stft(X))
        
        mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
        ext_features = mel
    
    except Exception as e:
        print("Error encountered while parsing file: ", file_name)
        return None 
     
    return np.array(ext_features)

In [20]:
testFile = '1-3.wav'
testF2 = extract_features_spec(testFile)
testF2.shape


(128,)

**Creating the Models**

In [21]:
# import the necessary packages
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras.layers import Flatten
from keras.layers import Input
from keras.models import Model

def create_mlp():
    
    # Construct model 
    model = Sequential()

    model.add(Dense(256, input_shape=(277,)))    ### 256
    model.add(Activation('relu'))
    model.add(Dropout(0.5))

#     model.add(Dense(512))
#     model.add(Activation('relu'))
#     model.add(Dropout(0.5))
    
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))

    #model.add(Dense(10))             ### 10 Classes
    #model.add(Activation('softmax'))

    # return our model
    return model

def create_cnn(width, height, depth, filters=(16, 32, 64)):

    # First CONV layer
    inputShape = (height, width, depth)
    inputs = Input(shape=inputShape)
    x = inputs
    x = Conv2D(32, (3, 3), padding="same")(x)
    x = Activation("relu")(x)
    
    
    # 2nd CONV
    x = Conv2D(64, (3, 3))(x)
    x = Activation("relu")(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.25)(x)
    
    
    # 3rd CONV
    x = Conv2D(64, (3, 3), padding="same")(x)
    x = Activation("relu")(x)
    
    
    # 4th CONV
    x = Conv2D(64, (3, 3))(x)
    x = Activation("relu")(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.5)(x)
    
    
    # 5th CONV
    x = Conv2D(128, (3, 3), padding="same")(x)
    x = Activation("relu")(x)
    
    
    # 6th CONV
    x = Conv2D(128, (3, 3))(x)
    x = Activation("relu")(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.7)(x)                             #0.5
    
    
    # Flatten and Output
    x = Flatten()(x)
    x = Dense(512)(x)
    x = Activation("relu")(x)
    x = Dropout(0.7)(x)                             #0.5
    
    
    # Output Layer (10 Classes)
    #x = Dense(10)(x)
    #x = Activation('softmax')(x)

    # construct the CNN
    model = Model(inputs, x)
    
    # return the CNN
    return model

## Data Preparation

**Audio Features: Train**

In [22]:
#alldf=pd.read_csv('all.csv',dtype=str)
#audioPath = "all/"                          ##### for AUDIO data

traindf = pd.read_csv('train.csv',dtype=str)###記得改
trainAudioPath = ""###記得改

In [23]:
# MFCCs Features
# Metadata file & full UrbanSound dataset 

features = []
print("[INFO] Extracting Features...")
for fileID in traindf["ID"]:
    audioFile = fileID + '.wav'
    audioDir = trainAudioPath + audioFile
    #print(audioPath)
    data = extract_features(audioDir)
    features.append(data)
    
print('Finished feature extraction from ', len(features), 'files') 

[INFO] Extracting Features...
Finished feature extraction from  280 files


In [24]:
# Save to binary file

np.save('trainAudioFeature277-1', features) 
print("The array is saved in the .npy file") 

The array is saved in the .npy file


In [25]:
features = np.array(features)
features.shape

(280, 277)

In [26]:
#features[0]

**Auido Features: Test**

In [28]:
testdf = pd.read_csv('test.csv', dtype=str)###記得改
testAudioPath = ""###記得改

In [29]:
# MFCCs Features
# Metadata file & full UrbanSound dataset 

test_features = []
print("[INFO] Extracting Features...")
for fileID in testdf["ID"]:
    audioFile = fileID + '.wav'
    audioDir = testAudioPath + audioFile
    #print(audioPath)
    data = extract_features(audioDir)
    test_features.append(data)
    
print('Finished feature extraction from ', len(test_features), 'files') 

[INFO] Extracting Features...
Finished feature extraction from  120 files


In [30]:
# Save to binary file

np.save('testAudioFeature277-1', test_features) 
print("The array is saved in the .npy file") 

The array is saved in the .npy file


**Load Audio Features**

In [31]:
trainAudioFeatures = np.load('trainAudioFeature277-1.npy', allow_pickle=True)
testAudioFeatures = np.load('testAudioFeature277-1.npy', allow_pickle=True)

In [32]:
trainAudioFeatures = np.array(trainAudioFeatures)
testAudioFeatures = np.array(testAudioFeatures)

In [33]:
import sklearn as sk 
scaler1 = sk.preprocessing.StandardScaler().fit(trainAudioFeatures)
trainAudioFeatures = scaler1.transform(trainAudioFeatures)
testAudioFeatures = scaler1.transform(testAudioFeatures)

**Labels**

In [34]:
#from keras.utils.np_utils import to_categorical
# LABELS
train_labels = traindf["Class"]
test_labels = testdf["Class"]
#labels = to_categorical(labels, 10)

In [35]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

le.fit(train_labels)
train_labels = le.transform(train_labels)
test_labels = le.transform(test_labels)

In [36]:
train_labels

array([5, 3, 9, 3, 5, 6, 0, 1, 4, 7, 2, 0, 7, 6, 6, 0, 9, 3, 7, 1, 5, 8,
       7, 6, 5, 8, 2, 1, 4, 6, 4, 5, 8, 2, 9, 1, 3, 4, 4, 2, 2, 8, 2, 8,
       3, 7, 7, 1, 7, 7, 2, 8, 5, 6, 6, 2, 7, 0, 8, 1, 5, 6, 5, 5, 5, 0,
       6, 7, 3, 6, 2, 0, 9, 3, 8, 2, 0, 8, 6, 7, 5, 2, 5, 4, 0, 0, 1, 1,
       9, 5, 9, 0, 4, 4, 3, 8, 7, 3, 4, 6, 5, 2, 6, 1, 9, 6, 3, 6, 4, 7,
       5, 1, 1, 9, 2, 9, 9, 9, 2, 1, 8, 1, 2, 4, 2, 3, 3, 1, 6, 5, 9, 0,
       7, 1, 0, 2, 4, 9, 7, 7, 9, 1, 8, 1, 4, 9, 0, 1, 2, 7, 3, 8, 2, 2,
       3, 1, 0, 4, 1, 3, 7, 3, 1, 9, 8, 4, 8, 0, 3, 0, 3, 4, 4, 5, 5, 3,
       4, 6, 2, 0, 5, 5, 6, 1, 8, 3, 3, 7, 8, 9, 9, 6, 7, 0, 8, 7, 1, 6,
       0, 1, 6, 8, 0, 0, 8, 2, 6, 4, 4, 8, 9, 1, 2, 8, 3, 5, 3, 4, 7, 9,
       0, 7, 4, 3, 4, 4, 8, 0, 5, 7, 8, 2, 4, 5, 7, 1, 7, 1, 9, 2, 2, 9,
       2, 9, 5, 6, 8, 6, 3, 5, 3, 2, 5, 3, 9, 6, 7, 9, 9, 5, 0, 9, 1, 0,
       6, 6, 7, 6, 0, 5, 0, 3, 8, 4, 4, 8, 4, 8, 0, 9])

### 1. Training MLP (1-D Feature)

In [37]:
from sklearn.model_selection import train_test_split
from keras.layers.core import Dense
from keras.models import Model
from keras.optimizers import Adam
from keras.layers import concatenate
import numpy as np
import argparse
import locale
import os
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, optimizers
from keras.utils.np_utils import to_categorical


print("[INFO] loading data...")
#features, images, labels = features, images, labels

print("[INFO] processing data...")
#split = train_test_split(features, images, labels, test_size=0.25, random_state=42)
#(trainFeatureX, testFeatureX, trainImagesX, testImagesX, trainY, testY) = split


trainFeatureX, testFeatureX, trainY, testY = trainAudioFeatures, testAudioFeatures, train_labels, test_labels

# For prediction and Confusion Matrix
trueY = testY

trainY = to_categorical(trainY, 10)###記得改
testY = to_categorical(testY, 10)###記得改


# create the MLP and CNN models
mlp = create_mlp()
#cnn = create_cnn(64, 64, 3)

# create the input to our final set of layers as the *output* of both the MLP and CNN
#combinedInput = concatenate([mlp.output, cnn.output])

Input = mlp.output

# The final FC layer head will have two dense layers, the final one being softmax layer
x = Dense(512, activation="relu")(Input)
x = Dense(10, activation="softmax")(x)###記得改

# The final model will accept audio on the MLP input and SPEC on the CNN input, outputting prediction
model = Model(inputs=mlp.input, outputs=x)

[INFO] loading data...
[INFO] processing data...



Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [38]:
trainFeatureX.shape

(280, 277)

In [39]:
trainFeatureX

array([[-1.02823775,  0.09519797,  0.72042677, ..., -0.1762181 ,
        -0.05608618, -0.72162112],
       [-0.70808832, -0.54849382, -1.1493005 , ..., -0.1743264 ,
         1.3426475 , -0.23376805],
       [-0.01900483,  1.12506077,  0.0637808 , ..., -0.17594879,
        -0.1873744 , -0.79054427],
       ...,
       [ 0.89016453,  1.32402243, -1.28461814, ..., -0.16836045,
         0.87325052,  0.96106802],
       [ 0.406394  , -1.02565322,  0.51445425, ..., -0.11388191,
         0.63058292, -0.13294628],
       [ 0.4693118 ,  0.64976905,  0.08967686, ..., -0.17439502,
        -1.07760843,  0.01542751]])

In [40]:
# Compile the model###可以改
model.compile(optimizers.rmsprop(lr=0.0005, decay=1e-6),loss="categorical_crossentropy",metrics=["accuracy"]) #lr 0.0005

# Train the model
print("[INFO] training model...")
history = model.fit(trainFeatureX, trainY, validation_data=(testFeatureX, testY), epochs=250, batch_size=32)  # 32


Instructions for updating:
keep_dims is deprecated, use keepdims instead

[INFO] training model...
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor

Train on 280 samples, validate on 120 samples
Epoch 1/250
280/280 [==============================] - 1s 4ms/step - loss: 2.2670 - acc: 0.2036 - val_loss: 1.6924 - val_acc: 0.7083
Epoch 2/250
280/280 [==============================] - 0s 100us/step - loss: 1.7275 - acc: 0.4286 - val_loss: 1.2618 - val_acc: 0.8417
Epoch 3/250
280/280 [==============================] - 0s 118us/step - loss: 1.3189 - acc: 0.6464 - val_loss: 0.8764 - val_acc: 0.9250
Epoch 4/250
280/280 [==============================] - 0s 125us/step - loss: 0.9713 - acc: 0.7536 - val_loss: 0.5936 - val_acc: 0.9500
Epoch 5/250
280/280 [==============================] - 0s 103us/step - loss: 0.7797 - acc: 0.7857 - val

280/280 [==============================] - 0s 103us/step - loss: 0.0157 - acc: 0.9964 - val_loss: 0.0195 - val_acc: 0.9917
Epoch 48/250
280/280 [==============================] - 0s 107us/step - loss: 0.0122 - acc: 0.9964 - val_loss: 0.0208 - val_acc: 0.9833
Epoch 49/250
280/280 [==============================] - 0s 100us/step - loss: 0.0572 - acc: 0.9929 - val_loss: 0.0125 - val_acc: 0.9917
Epoch 50/250
280/280 [==============================] - 0s 100us/step - loss: 0.0126 - acc: 0.9964 - val_loss: 0.0267 - val_acc: 0.9917
Epoch 51/250
280/280 [==============================] - 0s 107us/step - loss: 0.0378 - acc: 0.9786 - val_loss: 0.0358 - val_acc: 0.9833
Epoch 52/250
280/280 [==============================] - 0s 96us/step - loss: 0.0250 - acc: 0.9929 - val_loss: 0.0259 - val_acc: 0.9833
Epoch 53/250
280/280 [==============================] - 0s 100us/step - loss: 0.0112 - acc: 0.9964 - val_loss: 0.0413 - val_acc: 0.9917
Epoch 54/250
280/280 [==============================] - 0s 107

280/280 [==============================] - 0s 103us/step - loss: 0.0027 - acc: 1.0000 - val_loss: 0.0235 - val_acc: 0.9917
Epoch 108/250
280/280 [==============================] - 0s 103us/step - loss: 0.0049 - acc: 0.9964 - val_loss: 0.0242 - val_acc: 0.9917
Epoch 109/250
280/280 [==============================] - 0s 96us/step - loss: 0.0088 - acc: 0.9964 - val_loss: 0.0164 - val_acc: 0.9917
Epoch 110/250
280/280 [==============================] - 0s 103us/step - loss: 0.0074 - acc: 0.9964 - val_loss: 0.0279 - val_acc: 0.9917
Epoch 111/250
280/280 [==============================] - 0s 118us/step - loss: 0.0031 - acc: 1.0000 - val_loss: 0.0100 - val_acc: 0.9917
Epoch 112/250
280/280 [==============================] - 0s 121us/step - loss: 0.0046 - acc: 1.0000 - val_loss: 0.0226 - val_acc: 0.9917
Epoch 113/250
280/280 [==============================] - 0s 103us/step - loss: 0.0067 - acc: 0.9964 - val_loss: 0.0192 - val_acc: 0.9917
Epoch 114/250
280/280 [==============================] -

280/280 [==============================] - 0s 103us/step - loss: 0.0084 - acc: 0.9964 - val_loss: 0.0573 - val_acc: 0.9833
Epoch 167/250
280/280 [==============================] - 0s 96us/step - loss: 0.0027 - acc: 1.0000 - val_loss: 0.0451 - val_acc: 0.9833
Epoch 168/250
280/280 [==============================] - 0s 103us/step - loss: 9.8482e-04 - acc: 1.0000 - val_loss: 0.0388 - val_acc: 0.9833
Epoch 169/250
280/280 [==============================] - 0s 96us/step - loss: 0.0079 - acc: 0.9964 - val_loss: 0.0528 - val_acc: 0.9750
Epoch 170/250
280/280 [==============================] - 0s 103us/step - loss: 0.0282 - acc: 0.9893 - val_loss: 0.0656 - val_acc: 0.9750
Epoch 171/250
280/280 [==============================] - 0s 103us/step - loss: 0.0022 - acc: 1.0000 - val_loss: 0.0523 - val_acc: 0.9833
Epoch 172/250
280/280 [==============================] - 0s 107us/step - loss: 0.0011 - acc: 1.0000 - val_loss: 0.0473 - val_acc: 0.9833
Epoch 173/250
280/280 [==============================

280/280 [==============================] - 0s 103us/step - loss: 8.3887e-04 - acc: 1.0000 - val_loss: 0.0366 - val_acc: 0.9833
Epoch 226/250
280/280 [==============================] - 0s 103us/step - loss: 4.8268e-04 - acc: 1.0000 - val_loss: 0.0461 - val_acc: 0.9833
Epoch 227/250
280/280 [==============================] - 0s 103us/step - loss: 8.0601e-04 - acc: 1.0000 - val_loss: 0.0462 - val_acc: 0.9833
Epoch 228/250
280/280 [==============================] - 0s 107us/step - loss: 0.0018 - acc: 1.0000 - val_loss: 0.0641 - val_acc: 0.9833
Epoch 229/250
280/280 [==============================] - 0s 100us/step - loss: 8.6602e-04 - acc: 1.0000 - val_loss: 0.0486 - val_acc: 0.9833
Epoch 230/250
280/280 [==============================] - 0s 103us/step - loss: 1.7099e-04 - acc: 1.0000 - val_loss: 0.0400 - val_acc: 0.9833
Epoch 231/250
280/280 [==============================] - 0s 107us/step - loss: 0.0067 - acc: 0.9929 - val_loss: 0.0481 - val_acc: 0.9833
Epoch 232/250
280/280 [============

In [41]:
# Make predictions
print("[INFO] Evaluating...")
model.evaluate(testFeatureX, testY)

[INFO] Evaluating...
120/120 [==============================] - 0s 58us/step


[0.015788771898951382, 0.9916666666666667]

## 2. Training CNN (1-D Feature)

In [42]:
from sklearn.model_selection import train_test_split
from keras.layers.core import Dense
from keras.models import Model
from keras.optimizers import Adam
from keras.layers import concatenate
import numpy as np
import argparse
import locale
import os
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, Conv1D, MaxPool1D
from keras import regularizers, optimizers
from keras.utils.np_utils import to_categorical


print("[INFO] loading data...")
#features, images, labels = features, images, labels

print("[INFO] processing data...")
#split = train_test_split(features, images, labels, test_size=0.25, random_state=42)
#(trainFeatureX, testFeatureX, trainImagesX, testImagesX, trainY, testY) = split


trainFeatureX, testFeatureX, trainY, testY = trainAudioFeatures, testAudioFeatures, train_labels, test_labels

# For prediction and Confusion Matrix
trueY = testY

trainY = to_categorical(trainY, 10)####記得改
testY = to_categorical(testY, 10)####記得改


# define model
n_steps = 1
n_features = 277 ###記得改

# model = Sequential()
# model.add(Conv1D(filters=64, kernel_size=1, activation='relu', input_shape=(n_steps, n_features)))
# model.add(MaxPooling1D(pool_size=2))
# model.add(Flatten())
# model.add(Dense(50, activation='relu'))
# model.add(Dense(16,activation="softmax"))
# model.compile(optimizer='adam', loss='mse')


model = Sequential()
model.add(Conv1D(nb_filter=512, filter_length=1, input_shape=(n_features, 1)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dropout(0.4))
model.add(Dense(1024, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(10)) ####記得改
model.add(Activation('softmax'))

model.summary()

# # create the MLP and CNN models
# #mlp = create_mlp()
# cnn = create_cnn(120, 1, 1)

# # create the input to our final set of layers as the *output* of both the MLP and CNN
# #combinedInput = concatenate([mlp.output, cnn.output])

# Input = cnn.output

# # The final FC layer head will have two dense layers, the final one being softmax layer
# x = Dense(512, activation="relu")(Input)
# x = Dense(16, activation="softmax")(x)

# # The final model will accept audio on the MLP input and SPEC on the CNN input, outputting prediction
# model_cnn = Model(inputs=cnn.input, outputs=x)

[INFO] loading data...
[INFO] processing data...
Instructions for updating:
keep_dims is deprecated, use keepdims instead
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 277, 512)          1024      
_________________________________________________________________
activation_3 (Activation)    (None, 277, 512)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 141824)            0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 141824)            0         
_________________________________________________________________
dense_5 (Dense)              (None, 1024)              145228800 
_________________________________________________________________
dense_6 (Dense)              (None, 512)               524800    
____________________

C:\Users\Leo\Anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:48: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(input_shape=(277, 1), filters=512, kernel_size=1)`


In [46]:
# Compile the model###記得改###記得改###記得改###記得改
model.compile(optimizers.rmsprop(lr=0.0005, decay=1e-6),loss="categorical_crossentropy",metrics=["accuracy"]) #lr 0.0005

# Train the model
print("[INFO] training model...")
history = model.fit(trainFeatureX.reshape(280, 277,1), trainY, validation_data=(testFeatureX.reshape(120, 277,1), testY), epochs=100, batch_size=64)####記得改  # 32

[INFO] training model...
Train on 280 samples, validate on 120 samples
Epoch 1/100
280/280 [==============================] - 3s 9ms/step - loss: 0.0632 - acc: 0.9857 - val_loss: 0.4965 - val_acc: 0.9417
Epoch 2/100
280/280 [==============================] - 0s 1ms/step - loss: 0.0479 - acc: 0.9929 - val_loss: 0.1361 - val_acc: 0.9833
Epoch 3/100
280/280 [==============================] - 0s 1ms/step - loss: 3.6895e-06 - acc: 1.0000 - val_loss: 0.1339 - val_acc: 0.9833
Epoch 4/100
280/280 [==============================] - 0s 1ms/step - loss: 7.5745e-07 - acc: 1.0000 - val_loss: 0.1341 - val_acc: 0.9833
Epoch 5/100
280/280 [==============================] - 0s 1ms/step - loss: 6.5504e-07 - acc: 1.0000 - val_loss: 0.1349 - val_acc: 0.9750
Epoch 6/100
280/280 [==============================] - 0s 1ms/step - loss: 1.1788e-06 - acc: 1.0000 - val_loss: 0.1364 - val_acc: 0.9750
Epoch 7/100
280/280 [==============================] - 0s 1ms/step - loss: 2.3906e-07 - acc: 1.0000 - val_loss: 0.1

Epoch 60/100
280/280 [==============================] - 0s 1ms/step - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 0.1585 - val_acc: 0.9750
Epoch 61/100
280/280 [==============================] - 0s 1ms/step - loss: 1.1964e-07 - acc: 1.0000 - val_loss: 0.1598 - val_acc: 0.9750
Epoch 62/100
280/280 [==============================] - 0s 1ms/step - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 0.1598 - val_acc: 0.9750
Epoch 63/100
280/280 [==============================] - 0s 1ms/step - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 0.1598 - val_acc: 0.9750
Epoch 64/100
280/280 [==============================] - 0s 1ms/step - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 0.1598 - val_acc: 0.9750
Epoch 65/100
280/280 [==============================] - 0s 1ms/step - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 0.1598 - val_acc: 0.9750
Epoch 66/100
280/280 [==============================] - 0s 1ms/step - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 0.1598 - val_acc: 0.9750
Epoch 67/100
280/280 [============

In [44]:
\# Make predictions
print("[INFO] Evaluating...")
model.evaluate(testFeatureX.reshape(120,277,1), testY)####記得改

[INFO] Evaluating...
120/120 [==============================] - 0s 158us/step


[0.2514374089737733, 0.9583333293596904]

## 3. CNN (1-D Feature: Mel-Spec)

**Feature Extraction: Mean of Mel-Spec**

In [47]:
traindf = pd.read_csv('train.csv',dtype=str)
trainAudioPath = ""

mel_features = []
print("[INFO] Extracting Features...")
for fileID in traindf["ID"]:
    audioFile = fileID + '.wav'
    audioDir = trainAudioPath + audioFile
    #print(audioPath)
    data = extract_features_spec(audioDir)
    mel_features.append(data)
    
print('Finished feature extraction from ', len(mel_features), 'files') 

# Save to binary file
np.save('trainMelFeature', mel_features) 
print("The array is saved in the .npy file") 

[INFO] Extracting Features...
Finished feature extraction from  280 files
The array is saved in the .npy file


In [48]:
testdf = pd.read_csv('test.csv',dtype=str)
trainAudioPath = ""

mel2_features = []
print("[INFO] Extracting Features...")
for fileID in testdf["ID"]:
    audioFile = fileID + '.wav'
    audioDir = trainAudioPath + audioFile
    #print(audioPath)
    data = extract_features_spec(audioDir)
    mel2_features.append(data)
    
print('Finished feature extraction from ', len(mel2_features), 'files') 

# Save to binary file
np.save('testMelFeature', mel2_features) 
print("The array is saved in the .npy file") 

[INFO] Extracting Features...
Finished feature extraction from  120 files
The array is saved in the .npy file


In [49]:
import sklearn as sk 

trainMelFeatures = np.load('trainMelFeature.npy', allow_pickle=True)
testMelFeatures = np.load('testMelFeature.npy', allow_pickle=True)

trainMelFeatures = np.array(trainMelFeatures)
testMelFeatures = np.array(testMelFeatures)

scaler1 = sk.preprocessing.StandardScaler().fit(trainMelFeatures)
trainMelFeatures = scaler1.transform(trainMelFeatures)
testMelFeatures = scaler1.transform(testMelFeatures)

In [50]:
trainMelFeatures.shape
testMelFeatures.shape

(120, 128)

In [55]:
from sklearn import preprocessing

train_labels = traindf["Class"]
test_labels = testdf["Class"]

le = preprocessing.LabelEncoder()
le.fit(train_labels)
train_labels = le.transform(train_labels)
test_labels = le.transform(test_labels)

In [56]:
from sklearn.model_selection import train_test_split
from keras.layers.core import Dense
from keras.models import Model
from keras.optimizers import Adam
from keras.layers import concatenate
import numpy as np
import argparse
import locale
import os
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, Conv1D, MaxPool1D
from keras import regularizers, optimizers
from keras.utils.np_utils import to_categorical


print("[INFO] loading data...")
#features, images, labels = features, images, labels

print("[INFO] processing data...")
#split = train_test_split(features, images, labels, test_size=0.25, random_state=42)
#(trainFeatureX, testFeatureX, trainImagesX, testImagesX, trainY, testY) = split


trainFeatureX, testFeatureX, trainY, testY = trainMelFeatures, testMelFeatures, train_labels, test_labels

# For prediction and Confusion Matrix
trueY = testY

trainY = to_categorical(trainY, 10)###記得改
testY = to_categorical(testY, 10)###記得改


# define model
n_steps = 1
n_features = 128###記得改

model = Sequential()
model.add(Conv1D(nb_filter=512, filter_length=1, input_shape=(n_features, 1)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dropout(0.4))
model.add(Dense(2048, activation='relu'))
model.add(Dense(1024, activation='relu'))
model.add(Dense(10))####記得改
model.add(Activation('softmax'))###記得改

[INFO] loading data...
[INFO] processing data...


C:\Users\Leo\Anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:39: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(input_shape=(128, 1), filters=512, kernel_size=1)`


In [57]:
# Compile the model###記得改
model.compile(optimizers.rmsprop(lr=0.0005, decay=1e-6),loss="categorical_crossentropy",metrics=["accuracy"]) #lr 0.0005

# Train the model
print("[INFO] training model...")
history = model.fit(trainFeatureX.reshape(280, 128,1), trainY, validation_data=(testFeatureX.reshape(120, 128,1), testY), epochs=250, batch_size=32)  # 32

[INFO] training model...
Train on 280 samples, validate on 120 samples
Epoch 1/250
280/280 [==============================] - 3s 11ms/step - loss: 3.8090 - acc: 0.2143 - val_loss: 1.9229 - val_acc: 0.4333
Epoch 2/250
280/280 [==============================] - 0s 2ms/step - loss: 1.2678 - acc: 0.6429 - val_loss: 1.2317 - val_acc: 0.5917
Epoch 3/250
280/280 [==============================] - 0s 2ms/step - loss: 0.6391 - acc: 0.8607 - val_loss: 1.1807 - val_acc: 0.6333
Epoch 4/250
280/280 [==============================] - 0s 2ms/step - loss: 0.5370 - acc: 0.8214 - val_loss: 0.9778 - val_acc: 0.7083
Epoch 5/250
280/280 [==============================] - 0s 2ms/step - loss: 0.5343 - acc: 0.8750 - val_loss: 1.3244 - val_acc: 0.6333
Epoch 6/250
280/280 [==============================] - 0s 2ms/step - loss: 0.2948 - acc: 0.9321 - val_loss: 0.6096 - val_acc: 0.7917
Epoch 7/250
280/280 [==============================] - 0s 2ms/step - loss: 0.1300 - acc: 0.9536 - val_loss: 1.2474 - val_acc: 0.66

280/280 [==============================] - 0s 2ms/step - loss: 0.0576 - acc: 0.9964 - val_loss: 1.1827 - val_acc: 0.8583
Epoch 62/250
280/280 [==============================] - 0s 2ms/step - loss: 0.0576 - acc: 0.9964 - val_loss: 1.1796 - val_acc: 0.8583
Epoch 63/250
280/280 [==============================] - 0s 2ms/step - loss: 0.0576 - acc: 0.9964 - val_loss: 1.1784 - val_acc: 0.8583
Epoch 64/250
280/280 [==============================] - 0s 2ms/step - loss: 0.0576 - acc: 0.9964 - val_loss: 1.1690 - val_acc: 0.8583
Epoch 65/250
280/280 [==============================] - 0s 2ms/step - loss: 0.0576 - acc: 0.9964 - val_loss: 1.1749 - val_acc: 0.8583
Epoch 66/250
280/280 [==============================] - 0s 2ms/step - loss: 0.0576 - acc: 0.9964 - val_loss: 1.1745 - val_acc: 0.8583
Epoch 67/250
280/280 [==============================] - 0s 2ms/step - loss: 0.0576 - acc: 0.9964 - val_loss: 1.1699 - val_acc: 0.8583
Epoch 68/250
280/280 [==============================] - 0s 2ms/step - loss:

280/280 [==============================] - 0s 2ms/step - loss: 1.4241e-07 - acc: 1.0000 - val_loss: 1.6039 - val_acc: 0.8167
Epoch 122/250
280/280 [==============================] - 0s 2ms/step - loss: 1.2794e-07 - acc: 1.0000 - val_loss: 1.5993 - val_acc: 0.8167
Epoch 123/250
280/280 [==============================] - 0s 2ms/step - loss: 1.2581e-07 - acc: 1.0000 - val_loss: 1.6090 - val_acc: 0.8167
Epoch 124/250
280/280 [==============================] - 0s 2ms/step - loss: 1.2411e-07 - acc: 1.0000 - val_loss: 1.6161 - val_acc: 0.8167
Epoch 125/250
280/280 [==============================] - 0s 2ms/step - loss: 1.2176e-07 - acc: 1.0000 - val_loss: 1.6142 - val_acc: 0.8167
Epoch 126/250
280/280 [==============================] - 0s 2ms/step - loss: 1.2219e-07 - acc: 1.0000 - val_loss: 1.6173 - val_acc: 0.8167
Epoch 127/250
280/280 [==============================] - 0s 2ms/step - loss: 1.2176e-07 - acc: 1.0000 - val_loss: 1.6159 - val_acc: 0.8167
Epoch 128/250
280/280 [==================

280/280 [==============================] - 0s 2ms/step - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 1.6831 - val_acc: 0.8250
Epoch 181/250
280/280 [==============================] - 0s 2ms/step - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 1.6831 - val_acc: 0.8250
Epoch 182/250
280/280 [==============================] - 0s 2ms/step - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 1.6831 - val_acc: 0.8250
Epoch 183/250
280/280 [==============================] - 0s 2ms/step - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 1.6831 - val_acc: 0.8250
Epoch 184/250
280/280 [==============================] - 0s 2ms/step - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 1.6831 - val_acc: 0.8250
Epoch 185/250
280/280 [==============================] - 0s 2ms/step - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 1.6831 - val_acc: 0.8250
Epoch 186/250
280/280 [==============================] - 0s 2ms/step - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 1.6831 - val_acc: 0.8250
Epoch 187/250
280/280 [==================

280/280 [==============================] - 0s 2ms/step - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 1.6889 - val_acc: 0.8167
Epoch 240/250
280/280 [==============================] - 0s 2ms/step - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 1.6889 - val_acc: 0.8167
Epoch 241/250
280/280 [==============================] - 0s 2ms/step - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 1.6889 - val_acc: 0.8167
Epoch 242/250
280/280 [==============================] - 0s 2ms/step - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 1.6889 - val_acc: 0.8167
Epoch 243/250
280/280 [==============================] - 0s 2ms/step - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 1.6889 - val_acc: 0.8167
Epoch 244/250
280/280 [==============================] - 0s 2ms/step - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 1.6889 - val_acc: 0.8167
Epoch 245/250
280/280 [==============================] - 0s 2ms/step - loss: 1.1921e-07 - acc: 1.0000 - val_loss: 1.6889 - val_acc: 0.8167
Epoch 246/250
280/280 [==================

In [58]:
# Make predictions
print("[INFO] Evaluating...")
model.evaluate(testFeatureX.reshape(120,128,1), testY)

[INFO] Evaluating...
120/120 [==============================] - 0s 116us/step


[1.6889240503311158, 0.8166666626930237]